In [1]:
import pandas as pd
import numpy as np

import json
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb
from lightgbm import LGBMRegressor

In [2]:
train_data = pd.read_csv('train_modified.csv') # Important!!
print(train_data.shape)
train_data.head()

/home/tintash/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(903653, 56)


,Unnamed: 0,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,...,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,...,NaN,NaN,NaN,(not set),NaN,NaN,google + online,organic,NaN,google
4,4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,...,NaN,NaN,NaN,(not set),NaN,True,(not provided),organic,NaN,google


In [3]:
#Print all the column names
print(train_data.columns.values)

['Unnamed: 0' 'channelGrouping' 'date' 'fullVisitorId' 'sessionId'
 'socialEngagementType' 'visitId' 'visitNumber' 'visitStartTime'
 'device.browser' 'device.browserSize' 'device.browserVersion'
 'device.deviceCategory' 'device.flashVersion' 'device.isMobile'
 'device.language' 'device.mobileDeviceBranding' 'device.mobileDeviceInfo'
 'device.mobileDeviceMarketingName' 'device.mobileDeviceModel'
 'device.mobileInputSelector' 'device.operatingSystem'
 'device.operatingSystemVersion' 'device.screenColors'
 'device.screenResolution' 'geoNetwork.city' 'geoNetwork.cityId'
 'geoNetwork.continent' 'geoNetwork.country' 'geoNetwork.latitude'
 'geoNetwork.longitude' 'geoNetwork.metro' 'geoNetwork.networkDomain'
 'geoNetwork.networkLocation' 'geoNetwork.region'
 'geoNetwork.subContinent' 'totals.bounces' 'totals.hits'
 'totals.newVisits' 'totals.pageviews' 'totals.transactionRevenue'
 'totals.visits' 'trafficSource.adContent'
 'trafficSource.adwordsClickInfo.adNetworkType'
 'trafficSource.adwordsC

In [4]:
relevantAttributes = ['totals.hits', 'visitNumber', 'totals.pageviews', 'visitStartTime',
                      'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.city',
                      'device.operatingSystem', 'device.browser',
                      'channelGrouping',
                      'totals.transactionRevenue']

train_data['totals.transactionRevenue'] = train_data['totals.transactionRevenue'].fillna(0)
train_data['totals.pageviews'] = train_data['totals.pageviews'].fillna(1)

trainData = train_data.sample(frac=0.8,random_state=200)
testData = train_data.drop(trainData.index)

trainData_LGB = train_data[relevantAttributes]
testData_LGB = testData[relevantAttributes]

trainData_LGB = pd.DataFrame(trainData_LGB)
testData_LGB = pd.DataFrame(testData_LGB)

In [5]:
#Figure out labeling for country param
sortedContinent = train_data.groupby('geoNetwork.continent')['totals.transactionRevenue'].mean()
sortedContinent = sortedContinent.sort_values(ascending=False)
continentCodeDict = {}
i = 0
for continent in sortedContinent.index:
    continentCodeDict[continent] = i
    i += 1
    
sortedCountries = train_data.groupby('geoNetwork.country')['totals.transactionRevenue'].mean()
sortedCountries = sortedCountries.sort_values(ascending=False)
countryCodeDict = {}
i = 0
for country in sortedCountries.index:
    countryCodeDict[country] = i
    i += 1
    
sortedCities = train_data.groupby('geoNetwork.city')['totals.transactionRevenue'].mean()
sortedCities = sortedCities.sort_values(ascending=False)
cityCodeDict = {}
i = 0
for city in sortedCities.index:
    cityCodeDict[city] = i
    i += 1
    
sortedDevices = train_data.groupby('device.operatingSystem')['totals.transactionRevenue'].mean()
sortedDevices = sortedDevices.sort_values(ascending=False)
deviceCodeDict = {}
i = 0
for device in sortedDevices.index:
    deviceCodeDict[device] = i
    i += 1
    
sortedBrowser = train_data.groupby('device.browser')['totals.transactionRevenue'].mean()
sortedBrowser = sortedBrowser.sort_values(ascending=False)
browserCodeDict = {}
i = 0
for browser in sortedBrowser.index:
    browserCodeDict[browser] = i
    i += 1

sortedChannelGrouping = train_data.groupby('channelGrouping')['totals.transactionRevenue'].mean()
sortedChannelGrouping = sortedChannelGrouping.sort_values(ascending=False)
channelGroupingCodeDict = {}
i = 0
for device in sortedChannelGrouping.index:
    channelGroupingCodeDict[device] = i
    i += 1

print(channelGroupingCodeDict)
print(deviceCodeDict)
print(browserCodeDict)
print(continentCodeDict)
#print(cityCodeDict)

{'Display': 0, 'Referral': 1, 'Direct': 2, 'Paid Search': 3, 'Organic Search': 4, '(Other)': 5, 'Affiliates': 6, 'Social': 7}
{'Chrome OS': 0, 'Macintosh': 1, 'Linux': 2, 'Windows': 3, 'Android': 4, 'iOS': 5, 'Windows Phone': 6, 'NTT DoCoMo': 7, 'BlackBerry': 8, 'Firefox OS': 9, 'FreeBSD': 10, 'Nintendo 3DS': 11, 'Xbox': 12, 'Nintendo Wii': 13, 'Nintendo WiiU': 14, 'Nokia': 15, 'OpenBSD': 16, 'Samsung': 17, 'SunOS': 18, '(not set)': 19}
{'Firefox': 0, 'Chrome': 1, 'Edge': 2, 'Internet Explorer': 3, 'Safari': 4, 'Amazon Silk': 5, 'Opera': 6, 'Safari (in-app)': 7, 'Android Webview': 8, 'DASH_JR_3G': 9, 'Konqueror': 10, 'Iron': 11, 'IE with Chrome Frame': 12, 'Hisense M20-M_LTE': 13, 'HTC802t_TD': 14, 'DoCoMo': 15, 'subjectAgent: NoticiasBoom': 16, 'Coc Coc': 17, 'LYF_LS_4002_12': 18, 'Changa 99695759': 19, 'CSM Click': 20, 'BlackBerry': 21, 'Apple-iPhone7C2': 22, 'Android Runtime': 23, 'Android Browser': 24, 'ADM': 25, '0': 26, 'LYF_LS_4002_11': 27, 'M5': 28, 'Lunascape': 29, 'Reddit': 3

In [7]:
trainY = trainData_LGB["totals.transactionRevenue"]
trainX = trainData_LGB[['totals.hits', 'visitNumber', 'totals.pageviews', 'visitStartTime',
                        'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.city', 
                        'device.operatingSystem', 'device.browser',
                        'channelGrouping']]

trainX['geoNetwork.continent'] = trainX['geoNetwork.continent'].transform(lambda x: continentCodeDict[x])
trainX['geoNetwork.country'] = trainX['geoNetwork.country'].transform(lambda x: countryCodeDict[x])
trainX['geoNetwork.city'] = trainX['geoNetwork.city'].transform(lambda x: cityCodeDict[x])
trainX['device.operatingSystem'] = trainX['device.operatingSystem'].transform(lambda x: deviceCodeDict[x])
trainX['device.browser'] = trainX['device.browser'].transform(lambda x: browserCodeDict[x])
trainX['channelGrouping'] = trainX['channelGrouping'].transform(lambda x: channelGroupingCodeDict[x])

testY = testData_LGB["totals.transactionRevenue"]
testX = testData_LGB[['totals.hits', 'visitNumber', 'totals.pageviews', 'visitStartTime',
                      'geoNetwork.continent','geoNetwork.country', 'geoNetwork.city',
                            'device.operatingSystem', 'device.browser',
                      'channelGrouping']]

testX['geoNetwork.continent'] = testX['geoNetwork.continent'].transform(lambda x: continentCodeDict[x])
testX['geoNetwork.country'] = testX['geoNetwork.country'].transform(lambda x: countryCodeDict[x])
testX['geoNetwork.city'] = testX['geoNetwork.city'].transform(lambda x: cityCodeDict[x])
testX['device.operatingSystem'] = testX['device.operatingSystem'].transform(lambda x: deviceCodeDict[x])
testX['device.browser'] = testX['device.browser'].transform(lambda x: browserCodeDict[x])
testX['channelGrouping'] = testX['channelGrouping'].transform(lambda x: channelGroupingCodeDict[x])


/home/tintash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/tintash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/tintash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org

In [8]:
df = trainData_LGB
for df in [trainX, testX]:
    df['date'] = pd.to_datetime(df['visitStartTime'], unit='s')
    df['sess_date_dow'] = df['date'].dt.dayofweek
    df['sess_date_hours'] = df['date'].dt.hour
    df['sess_date_dom'] = df['date'].dt.day
    df['date'] = df['date'].dt.date
    
trainX = trainX.drop(['date'], axis=1)
testX = testX.drop(['date'], axis=1)

In [9]:
# trainY = np.log1p(trainY.astype(float))
train_x, val_x, train_y, val_y = train_test_split(trainX, trainY, test_size=0.25, random_state=20)

# Feature Scaling
sc = StandardScaler()
train_x = sc.fit_transform(train_x)
testX = sc.transform(testX)

In [40]:
# cat_cols = ['geoNetwork.country', 'geoNetwork.city',
#                       'device.operatingSystem', 'device.browser',
#                       'channelGrouping']
# categorical_feature = [trainX[cat_cols].columns.get_loc(c) for c in cat_cols ]
# categorical_feature

[0, 1, 2, 3, 4]

In [10]:
#Initialize LGBM
reg = LGBMRegressor(objective = 'regression', 
                     boosting_type = 'gbdt', 
                     metric = 'rmse',
                     n_estimators = 20000, #10000
                     num_leaves = 36,
                     learning_rate = 0.03, #0.01
                     bagging_fraction = 0.9,#0.8
                     feature_fraction = 0.6,#.3
                     bagging_seed = 42,
                     max_depth = 12, #-1 
#                      categorical_feature = [train_x[cat_cols].columns.get_loc(c) for c in cat_cols ] ,
                     #cat_smooth = 20
                    )

reg.fit(
    train_x, np.log1p(train_y),
    eval_set=[(val_x, np.log1p(val_y))],
    early_stopping_rounds=50,
    verbose=100,
    eval_metric='rmse'
)

Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[17]	valid_0's rmse: 2.00686


LGBMRegressor(bagging_fraction=0.9, bagging_seed=42, boosting_type='gbdt',
       class_weight=None, colsample_bytree=1.0, feature_fraction=0.6,
       importance_type='split', learning_rate=0.03, max_depth=12,
       metric='rmse', min_child_samples=20, min_child_weight=0.001,
       min_split_gain=0.0, n_estimators=20000, n_jobs=-1, num_leaves=36,
       objective='regression', random_state=None, reg_alpha=0.0,
       reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0)

In [11]:
#Prediction
y_pred=reg.predict(testX , num_iteration=reg.best_iteration_)

In [12]:
y = testY

y_pred = y_pred + 1
y = y + 1

y_pred_ln = np.log(y_pred)
y_true_ln = np.log(y)

difference = np.subtract(y_pred_ln, y_true_ln)

sqr = np.square(difference)

sumSqr = sqr.sum()

sumSqrMean = sumSqr/len(y)

sumSqrRootMean = np.sqrt(sumSqrMean)
sumSqrRootMean

1.9166989680002817

In [ ]:
# params = {}
# params['learning_rate'] = 0.003
# params['boosting_type'] = 'gbdt'
# params['objective'] = 'binary'
# params['metric'] = 'binary_logloss'
# params['sub_feature'] = 0.5
# params['num_leaves'] = 10
# params['min_data'] = 50
# params['max_depth'] = 10